In [1]:
import numpy as np
import pandas as pd
from rubin_sim.maf.utils import getSimData
import sqlite3

In [2]:
import os
from sqlalchemy import create_engine
from sqlalchemy.engine import make_url
from sqlalchemy import inspect

__all__ = ["OpsimDatabase"]


class OpsimDatabase:
    """Stub for backwards compatibility"""

    def __init__(self, dbinfo, **kwargs):

        if "://" in dbinfo:
            # assume this is a user-configured engine connection string for sqlalchemy
            self.url = make_url(dbinfo)
        else:
            if not os.path.exists(dbinfo):
                FileExistsError(f"Cannot find existing database at {dbinfo}")
            self.url = make_url("sqlite:///" + dbinfo)
        # connection and save connection
        self.engine = create_engine(self.url)
        self.connection = self.engine.connect()
        # Find the default table name
        inspector = inspect(self.engine)
        tables = [inspector.get_table_names(schema=schema) for schema in inspector.get_schema_names()]
        # for our sqlite databases, there is only one schema
        if 'observations' in tables[0]:
            self.defaultTable = 'observations'
        elif 'summaryAllProps' in tables[0]:
            self.defaultTable = 'summaryallprops'
        elif 'summary' in tables[0]:
            self.defaultTable = 'summary'

In [3]:
# Does pd.read_sql have any preference about what the connection is?

In [4]:
connection = sqlite3.connect('baseline_v2.0_10yrs.db')

In [5]:
%%timeit
n = 25
for i in range(10):
    pd.read_sql(f'select fieldRA, fieldDec, night, filter, note from observations where night > {n+i*2} and night < {n + 320 + i*2}', 
                connection)

5.59 s ± 85.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
connection = create_engine('sqlite:///baseline_v2.0_10yrs.db')

In [7]:
%%timeit
n = 22
for i in range(10):
    pd.read_sql(f'select fieldRA, fieldDec, night, filter, note from observations where night > {n+i*2} and night < {n + 320 + i*2}', 
                connection)

6.09 s ± 88.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
connection = create_engine('sqlite:///baseline_v2.0_10yrs.db').connect()

In [9]:
%%timeit
n = 24
for i in range(10):
    pd.read_sql(f'select fieldRA, fieldDec, night, filter, note from observations where night > {n+i*2} and night < {n + 320 + i*2}', 
                connection)

6.03 s ± 41 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
# So .. sort of. I actually found these to vary a lot more than the standard deviation on reruns.

In [11]:
# Then - does it matter if we're opening and closing connections all the time? 

In [12]:
%%timeit
n = 80
for i in range(10):
    con = sqlite3.connect('baseline_v2.0_10yrs.db')
    data = getSimData(con, f'night > {i+n} and night < {i+n+320}', 
                      ['fieldRA', 'fieldDec', 'night', 'filter', 'note'], tableName='observations')
    con.close()

5.93 s ± 44 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%%timeit
n = 81
con = sqlite3.connect('baseline_v2.0_10yrs.db')
for i in range(10):
    data = getSimData(con, f'night > {i+n} and night < {i+n+320}', 
                      ['fieldRA', 'fieldDec', 'night', 'filter', 'note'], tableName='observations')
con.close()

6.23 s ± 227 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%%timeit
opsdb = OpsimDatabase('baseline_v2.0_10yrs.db')
n = 83
for i in range(10):
    data = getSimData(opsdb.connection, f'night > {n+i} and night < {n+i+320}', 
                      ['fieldRA', 'fieldDec', 'night', 'filter', 'note'], 
                      tableName=opsdb.defaultTable)

6.9 s ± 227 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
# And does it matter if we have to do extra work to figure out the table name each time?

In [16]:
%%timeit
n = 84
for i in range(10):
    data = getSimData('baseline_v2.0_10yrs.db', f'night > {i+n} and night < {i+n+320}', 
                      ['fieldRA', 'fieldDec', 'night', 'filter', 'note'], tableName='observations')

5.99 s ± 76 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%timeit
n = 85
for i in range(10):
    data = getSimData('baseline_v2.0_10yrs.db', f'night > {i+n} and night < {i+n+320}', 
                      ['fieldRA', 'fieldDec', 'night', 'filter', 'note'], tableName=None)

6.06 s ± 88.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
# .. mysterious. 